In [1]:
!pip install -U spacy==2.1.0
!python -m spacy download en

!pip install Cython --install-option="–no-cython-compile"


import nltk
nltk.download('punkt')
import pandas as pd
import spacy
nlp_pronoun = spacy.load('en')
import neuralcoref
neuralcoref.add_to_pipe(nlp_pronoun,greedyness=0.52)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-2.1.0-cp37-cp37m-manylinux1_x86_64.whl (27.7 MB)
  Using cached preshed-2.0.1-cp37-cp37m-manylinux1_x86_64.whl (82 kB)
  Using cached blis-0.2.4-cp37-cp37m-manylinux1_x86_64.whl (3.2 MB)
  Using cached thinc-7.0.8-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.8
    Uninstalling preshed-3.0.8:
      Successfully uninstalled preshed-3.0.8
  Attempting uninstall: blis
    Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.1
    Uninstalling thinc-7.4.1:
      Successfully uninstalled thinc-7.4.1
  Attempting uninstall: spacy
    Found existing installation: spacy 2.3.1
    Uninstalling spacy-2.3.1:
      Successfully uninstalled spacy-2.3.1
ERROR: pip's dependency reso

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
!pip install neuralcoref==4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install word2number

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5582 sha256=a1788fd66e3d6d7cdceaf0c6c05cbe16cabcf2f9667ae4d56fa50e79add7e5ae
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number


In [2]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
  import nltk
  nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
!pip install benepar



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!python -m spacy download en_core_web_md


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95.4 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [1]:
def replace(sentence):
  doc = nlp_pronoun(sentence)
  return doc._.coref_resolved

In [2]:
import benepar, spacy
#benepar.download('benepar_en3')
nlp = spacy.load('en_core_web_md')
if spacy.__version__.startswith('2'):
        nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
else:
        nlp.add_pipe("benepar", config={"model": "benepar_en3"})

In [3]:
def extract_sub_verb_object(sentence):
    subject = ""
    verb_phrase = ""
    preposition_phrase = ""
    #dependency_tree = sent1._.parse_string
    doc = nlp(sentence)
    sent = list(doc.sents)[0]
    dependency_tree = sent._.parse_string
    sb = 1
    vb = 0
    pb = 0

    for i in range(len(dependency_tree)):
      ch = dependency_tree[i]

      if sb == 1:
        if (ch.islower()==True or ch.isdigit()==True or ch==" " or ch=="," ):
          if( (subject[:-1]!=" " and ch==" ") or ch!=" "):
              subject = subject+ch

        
      
        elif ch=="V":
          sb = 0
          vb = 1
          continue

        
      if vb == 1:
        if (ch.islower()==True or ch.isdigit()==True or ch==" " or ch==","):
          if( (verb_phrase[:-1]!=" " and ch==" ") or ch!=" "):
            verb_phrase = verb_phrase+ch
          
        elif ch=="N" and dependency_tree[i+1] == "P":
          vb = 0
          pb = 1
          continue

      if pb == 1:
          if (ch.islower()==True or ch.isdigit()==True or ch==" " or ch==","):
            if( (preposition_phrase[:-1]!=" " and ch==" ") or ch!=" "):
              preposition_phrase = preposition_phrase+ch
          
        
         
      
    return {"subject":subject.strip(), "verb":verb_phrase.strip(), "object":preposition_phrase.strip()}

In [9]:
def split_conj(sent_list):

  sub = ""
  l = list()

  for i in sent_list:
    
    if i!="and":
      
        sub = sub+" "+i
       
 
    else:
      if sub!="":
        l.append(sub)
    
        sub = ""

  l.append(sub)


  return l

In [10]:
def handle_conjunction(sentence):
  d = extract_sub_verb_object(sentence)

  subject = d["subject"].replace(",","and")
  verb = d["verb"].replace(",","and")
  obj = d["object"].replace(",","and")
  subject_tokens = [i for i in subject.split(' ') if i != ' ']
  verb_tokens = [i for i in verb.split(' ') if i!= ' ']
  obj_tokens = [i for i in obj.split(' ') if i!=''] 
  sentences = list()
  if "and" in subject_tokens:
    split_subject = split_conj(subject_tokens)
    
    for i in split_subject:
      sentences.append(i+" "+verb+" "+obj)
    return sentences
  if "and" in obj_tokens:
    
    split_obj = split_conj(obj_tokens)
  
    
    for i in split_obj:
      sentences.append(subject+" " + verb+" "+ i)
  
    return sentences
  else:
    sentences.append(subject+" "+verb+" "+obj)
    return sentences

In [11]:
from nltk.tokenize import sent_tokenize
def split_mwp(mwp):
  res = []
  mwp_split_temp = sent_tokenize(mwp)
  mwp_split = list()
  for i in mwp_split_temp:
    temp = handle_conjunction(i)
    for j in temp:
      mwp_split.append(j.strip())
  for i,sent in enumerate(mwp_split):
    if "and" in sent:
      res.extend(handle_conjunction(sent))
      continue
    res.append(sent)
    
  return res

In [12]:
split_mwp("john and rebecca had 2 oranges.")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.7/dist-packages/torch/distributions/distribution.py:46: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  'with `validate_args=False` to turn off validation.')


['john  had 2  oranges', 'rebecca had 2  oranges']

In [8]:
!pip install checklist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.1 MB 5.1 MB/s 
     |████████████████████████████████| 6.4 MB 45.5 MB/s 
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 167 kB 61.1 MB/s 
  Using cached jedi-0.18.2-py2.py3-none-any.whl (1.6 MB)
  Using cached qtconsole-5.4.0-py3-none-any.whl (121 kB)
     |████████████████████████████████| 126 kB 64.4 MB/s 
     |████████████████████████████████| 806 kB 66.1 MB/s 
     |████████████████████████████████| 490 kB 66.1 MB/s 
     |████████████████████████████████| 10.2 MB 55.5 MB/s 
     |████████████████████████████████| 81 kB 11.0 MB/s 
     |████████████████████████████████| 5.6 MB 44.8 MB/s 
     |████████████████████████████████| 5.6 MB 48.8 MB/s 
     |████████████████████████████████| 348 kB 64.5 MB/s 
     |██████████████████████████

In [6]:
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 1.8 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047104 sha256=7e64c3951499371e134b34e7b7d653c66b484f7c8e5c1605fac2772ff6f4cf61
  Stored in directory: /tmp/pip-ephem-wheel-cache-t8ed89w9/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.1.0
    Uninstalling en-core-web-sm-2.1.0:
      Successfully uninstalled en-core-web-sm-2.1.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [14]:
!pip install spacy==2.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-2.3.1-cp37-cp37m-manylinux1_x86_64.whl (9.9 MB)
  Using cached blis-0.4.1-cp37-cp37m-manylinux1_x86_64.whl (3.7 MB)
  Using cached preshed-3.0.8-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (126 kB)
  Using cached thinc-7.4.1-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
  Attempting uninstall: preshed
    Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Attempting uninstall: blis
    Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Attempting uninstall: thinc
    Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Attempting uninstall: spacy
    Found existing installation: spacy 2.1.0
    Uninstalling spacy-2.1.0:
      S

In [1]:

import checklist
from checklist.editor import Editor

editor = Editor()

sample_test_data = editor.template(
    "{mask} and {mask} had 2 {mask} and 6 {mask}.", pos=["good", "great"], nsamples=10
).data

/usr/local/lib/python3.7/dist-packages/checklist/text_generation.py:171: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  to_pred = torch.tensor(to_pred, device=self.device).to(torch.int64)


In [4]:
sample_test_data

['Peter and Mary had 2 sons and 6 daughters.',
 'William and Kate had 2 sons and 6 daughters.',
 'William and Elizabeth had 2 boys and 6 girls.',
 'William and Mary had 2 daughters and 6 sons.',
 'Dad and Mom had 2 children and 6 grandchildren.',
 'Mom and Dad had 2 boys and 6 girls.',
 'William and Mary had 2 sons and 6 children.',
 'William and Elizabeth had 2 sons and 6 grandchildren.',
 'Mom and Dad had 2 brothers and 6 sisters.',
 'Mary and Joseph had 2 sons and 6 daughters.']

In [13]:
for i in sample_test_data:
  print(split_mwp(i.lower()))

['peter had  2 sons', 'peter had  6 daughters', 'mary had  2 sons', 'mary had  6 daughters']
['william had  2 sons', 'william had  6 daughters', 'kate had  2 sons', 'kate had  6 daughters']
['william had  2 boys', 'william had  6 girls', 'elizabeth had  2 boys', 'elizabeth had  6 girls']
['william had  2 daughters', 'william had  6 sons', 'mary had  2 daughters', 'mary had  6 sons']
['dad had  2 children', 'dad had  6 grandchildren', 'mom had  2 children', 'mom had  6 grandchildren']
['mom had  2 boys', 'mom had  6 girls', 'dad had  2 boys', 'dad had  6 girls']
['william had  2 sons', 'william had  6 children', 'mary had  2 sons', 'mary had  6 children']
['william had  2 sons', 'william had  6 grandchildren', 'elizabeth had  2 sons', 'elizabeth had  6 grandchildren']
['mom had  2 brothers', 'mom had  6 sisters', 'dad had  2 brothers', 'dad had  6 sisters']
['mary had  2 sons', 'mary had  6 daughters', 'joseph had  2 sons', 'joseph had  6 daughters']


In [14]:
sample_test_data_1 = editor.template(
    "there are 10 {mask} and 20 {mask} in Philip's collection.", nsamples=10
).data

In [15]:
sample_test_data_1

["there are 10 CDs and 20 tapes in Philip's collection.",
 "there are 10 books and 20 DVDs in Philip's collection.",
 "there are 10 albums and 20 films in Philip's collection.",
 "there are 10 books and 20 CDs in Philip's collection.",
 "there are 10 drawings and 20 photographs in Philip's collection.",
 "there are 10 albums and 20 singles in Philip's collection.",
 "there are 10 cats and 20 birds in Philip's collection.",
 "there are 10 letters and 20 notes in Philip's collection.",
 "there are 10 poems and 20 essays in Philip's collection.",
 "there are 10 CDs and 20 DVDs in Philip's collection."]

In [16]:
for i in sample_test_data_1:
  print(split_mwp(i.lower()))

['there are  10 cds', 'there are  20 tapes in philip s collection']
['there are  10 books', 'there are  20 dvds in philip s collection']
['there are  10 albums', 'there are  20 films in philip s collection']
['there are  10 books', 'there are  20 cds in philip s collection']
['there are  10 drawings', 'there are  20 photographs in philip s collection']
['there are  10 albums', 'there are  20 singles in philip s collection']
['there are  10 cats', 'there are  20 birds in philip s collection']
['there are  10 letters', 'there are  20 notes in philip s collection']
['there are  10 poems', 'there are  20 essays in philip s collection']
['there are  10 cds', 'there are  20 dvds in philip s collection']


In [17]:
sample_test_data_2 = editor.template(
    "{mask} received 10 {mask} and 3 {mask} and 4 {mask} in his {mask}", nsamples=10
).data
sample_test_data_2

['Williams received 10 points and 3 rebounds and 4 blocks in his debut',
 'Brown received 10 starts and 3 TDs and 4 interceptions in his career',
 'Walker received 10 tackles and 3 sacks and 4 interceptions in his debut',
 'Johnson received 10 points and 3 rebounds and 4 assists in his debut',
 'Williams received 10 points and 3 rebounds and 4 steals in his debut',
 'Brown received 10 starts and 3 touchdowns and 4 interceptions in his career',
 'Bush received 10 percent and 3 percent and 4 percent in his primary',
 'Wilson received 10 tackles and 3 sacks and 4 interceptions in his debut',
 'Brown received 10 starts and 3 starts and 4 losses in his career',
 'Hart received 10 starts and 3 wins and 4 losses in his career']

In [2]:
def extract_microstatements(sentence):
        """
        @input : one sentence at a time without coref resolution, because coref resolution ruins the question
        @output : {set of microstatements from a single sentence input}
        Performed using Parsing a dependency tree generated by benepar (berkeley neural parser)
        """
        # convert all to lower case, because the logic runs on the assumption that the labels are upper case and everything else is lower
        sentence = sentence.lower()
        parts = " "
        doc = nlp(sentence)
        try:
            #print(list(doc.sents))
            sent = list(doc.sents)[0]
        except:
            return []
        dependency_tree = sent._.parse_string
        #print(dependency_tree)
        dependency_tree = dependency_tree.replace("(", "( ")
        dependency_tree = dependency_tree.replace(")", " )")
        dependency_tree = dependency_tree.split()
        #splitting dependency tree such that each label, bracket, word is a separate elememt in the list
        i =0
        sentence_splits= [] #split sentences on the basis of presence of conjuction
        flag_complex_sentence = False #save time if the sentence is simple
        while i<len(dependency_tree):
            if dependency_tree[i] == "CC" or (dependency_tree[i] == "," and dependency_tree[i+1] == ','):
                flag_complex_sentence = True
                if parts != "":
                    sentence_splits.append(parts) #if a conjuction is encountered, a new part is added to the sentence
                parts= ""
                while(dependency_tree[i] != ")"):
                    i+=1
            elif dependency_tree[i].islower() or dependency_tree[i].isnumeric():
                parts = parts + dependency_tree[i] + " "
                i=i+1
            else:
                i+=1
        sentence_splits.append(parts)
        #print(sentence_splits)
        if flag_complex_sentence == False:
            return [sentence]
        phrase_checker= [] #to check whether a phrase can be an independent sentence
        #extract_noun = []
        flag_no_verb = False
        for i in sentence_splits:
            #creates a dependency tree for each part of the sentence
            doc = nlp(i)
            sent = list(doc.sents)[0]
            #print(sent)
            dependency_tree = sent._.parse_string
            dependency_tree = dependency_tree.replace("(", "( ")
            dependency_tree = dependency_tree.replace(")", " )")
            dependency_tree = dependency_tree.split()
            #print(dependency_tree)
            #if a verb is present in the phrase then it is an independent sentence
            if "VP" in dependency_tree:
                phrase_checker.append([True,i,dependency_tree])
                #flag_no_verb = False 
            else:
                phrase_checker.append([False,i])
                #print(i, dependency_tree)
                flag_no_verb = True
                break
            #tells us if even one phrase is incomplete/dependent

            # if "JJ" in dependency_tree:
            #     last_adj_idx = max(idx for idx, val in enumerate(dependency_tree) if val == 'JJ')
            #     j = last_adj_idx
            #     flag_adj = False
            #     while(j<len(dependency_tree)):
            #         if dependency_tree[j] == "NN" or dependency_tree[j] == "NNS" or dependency_tree[j] == "NNP" or dependency_tree[j] == "NNPS":
            #             extract_noun.append([i, dependency_tree[j+1]])
            #             flag_adj = True
            #         j+=1
            #     if flag_adj == False:
            #         extract_noun.append([i, ""])
        
                    

        #print(extract_noun)
        if flag_no_verb:
            #print("never here")
            common_phrases_ls = []
            #to find a common phrase that will complete an incomplete phrase
            for i in phrase_checker:
                if i[0]:
                    #print(i[1])
                    common_phrase = ""
                    last_verb_idx = max(idx for idx, val in enumerate(i[2]) if val == 'VP') #finds the last verb in an independent sentence, copies the sentence up until then
                    for j in i[2][:last_verb_idx]:
                        if j.islower() or j.isnumeric():
                            common_phrase = common_phrase + j + " " #
                    last_verb_idx +=2
                    brackets = 1
                    while brackets>0: #to include the final verb in the sentence
                        if i[2][last_verb_idx] == "(":
                            brackets+=1
                        elif i[2][last_verb_idx] == ")":
                            brackets-=1
                        elif i[2][last_verb_idx].islower() or i[2][last_verb_idx].isnumeric():
                            common_phrase = common_phrase + i[2][last_verb_idx] + " "
                        else:
                            pass
                        last_verb_idx+=1
                    common_phrases_ls.append(common_phrase)
            true_count = 0 #checks which common phrase is applicable to which incomplete phrase
            for i in phrase_checker: 
                if i[0]:
                    true_count+=1 
                if not i[0]: #for all phrases that are dependent, it adds the common phrase to the start of the phrase
                    try:
                        i[1] = common_phrases_ls[true_count-1] +" "+ i[1]
                    except:
                        i[1] = i[1]

        micros = []
        #appends all the found phrases to the list of microstatements
        for i in phrase_checker:
            micros.append(i[1])
        #print(micros)
        '''in case of a noun that proceeds an adjective, the noun does not get included into the microstatement.
        This is to append that extra noun in all sentences where it remains missing'''
        #print(micros)
        extract_noun = []
        for i in micros:
           # print(i)
            doc = nlp(i)
            sent = list(doc.sents)[0]
            dependency_tree = sent._.parse_string
            dependency_tree = dependency_tree.replace("(", "( ")
            dependency_tree = dependency_tree.replace(")", " )")
            dependency_tree = dependency_tree.split()
            #print(dependency_tree)
            if "JJ" in dependency_tree:
                #print(dependency_tree) #checks to see if this is a concern for us at all
                last_adj_idx = max(idx for idx, val in enumerate(dependency_tree) if val == 'JJ')
                j = last_adj_idx #finds the last adjective in the sentence
                #print(j)
                flag_adj = False
                temp_noun = ""
                while(j<len(dependency_tree)): #checks to see if there is a noun after the adjective
                    if dependency_tree[j] == "NN" or dependency_tree[j] == "NNS" or dependency_tree[j] == "NNP" or dependency_tree[j] == "NNPS":
                        temp_noun = temp_noun + dependency_tree[j+1] + " "
                        flag_adj = True
                    j+=1
                extract_noun.append([i,temp_noun])   
                
                if flag_adj == False:
                    extract_noun.append([i, ""]) #if there is no noun after the adjective, then the noun is missing and needs to be added
            else:
                extract_noun.append([i,"ignore"]) #if there is no adjective, then we don't need to worry about this sentence
        modified_micros = []
        #print(extract_noun)
        for i in range(len(extract_noun)):
            #print("extract_nouns", extract_noun[i])
            if extract_noun[i][1] !="": #if the noun is not empty, then we add it to the microstatements list, since it is a complete sentence
                #print("i was here")
                modified_micros.append(extract_noun[i][0])
            else:
                #print("i came here")
                temp = i+1
                while(temp<len(extract_noun)):
                    if extract_noun[temp][1] =="ignore": #sentences that don't have an adjective are ignored
                        temp+=1
                        continue
                    elif extract_noun[temp][1]=="": #if the next sentence also doesn't have a noun, then we ignore it
                        temp+=1
                        continue
                    else: #if the sentence has a noun, we extract that and add it to the current microstatement
                        temp_micro = extract_noun[i][0] + " " + extract_noun[temp][1]
                        modified_micros.append(temp_micro)
                        break
                    temp+=1
        return modified_micros

In [3]:
import nltk
import re
from word2number import w2n

    
def convertNumberNames(sentence):
    lstr = re.findall( r'\w+|[^\s\w]+', sentence)
    # print(lstr)
    res = []
    i = 0
    status = []
    # False if word isnt a number indicated by exception
    for i in range(len(lstr)):
        try:
            w2n.word_to_num(lstr[i])
            status.append(True)
        except:
            # for complex number names like "one hundred and thirty five"
            if lstr[i]=='and' and i>0 and status[i-1] == True:
                if i<len(lstr)-1:
                    try:
                        w2n.word_to_num(lstr[i+1])
                        status.append(True)
                    except:
                        status.append(False)
                else:
                    status.append(False)
            else:
                status.append(False)
    
    j = 0
    # print(status)
    # convert all the consecutive True's to a number eg. ten -> 10; thirty, five -> 35
    final_ls = []
    while j<len(lstr):
        quant = ""
        while(status[j]==True):
            quant = quant+lstr[j]+" "
            j=j+1
        if quant!="":
            try:
                final_ls.extend([str(w2n.word_to_num(quant)),lstr[j]])
            except:
                final_ls.extend([quant.strip(),lstr[j]])
        else:
            final_ls.append(lstr[j])
        j=j+1
    final_ls = " ".join(final_ls)
    return final_ls

def appendQuantities(txt):
    wordsList = nltk.word_tokenize(txt)
    #wordsList = [w for w in wordsList if not w in stop_words]
    
    tagged = nltk.pos_tag(wordsList)
    # determining the units in the word problem 
    
    unit = []
    for i in range(len(tagged)-2):
        #print(tagged[i])
        if tagged[i][1] == "CD":
            #print(tagged[i], "Here")
            j = i+1
            while(j<len(tagged)-1):
                #print(tagged[j])
                if (tagged[j][1]=="NN" or tagged[j][1]=="NNS"):
                    #print(tagged[i][0],"HERE")
                    unit.append([tagged[j][0], j])
                    #print("Unit", unit)
                    break
                else:
                    j=j+1
                    break
    #appending units
    #print(unit) 
    unit_count = 0
    new_mwp = ""
    for i in range(len(tagged)):
        if unit_count<len(unit)-1 and unit[unit_count][1] == i :
            unit_count+=1
        if (tagged[i][1] == 'CD' and tagged[i+1][1]!='NN' and tagged[i+1][1]!='NNS'):
                #tagged.insert(i+1,unit[unit_count])
                new_mwp = new_mwp + tagged[i][0] + " " + unit[unit_count][0] + " "
        else:
            new_mwp = new_mwp + tagged[i][0] + " "
    
    return new_mwp

    # entities = list()
    # s =""
    # #extracting quantities
    # for i in range(len(tagged)-2):
    #     if tagged[i][1] == "CD":
    #         #if tagged[i+1][1] == "NN" or tagged[i+1][1]=='NNP'or tagged[i+1][1]=='NNPS' or tagged[i+2][1]=='NN' or tagged[i+2][1]=='NNP'or tagged[i+2][1]=='NNPS' or  tagged[i+1][1]=='NNS' or tagged[i+2][1]=='NNS' :
    #         if tagged[i+1][1] == "NN" or tagged[i+2][1]=='NN' or tagged[i+1][1]=='NNS' or tagged[i+2][1]=='NNS' :
    #             s = tagged[i][0] + " "+ tagged[i+1][0]
    #             entities.append(s)
    
    # return(entities)
#print(convertNumberNames("Joe has five and amy has 6 apples"))

In [4]:
def identify_question(microstatements):
    # microstatements = mwp.split(".") ##microstatements module fits here, for now i have simply split sentence at full stop
    statements = []
    question = []
    question_words = ["evaluate", "calculate", "find", "determine","how", "what","identify"]
    for i in microstatements:
        flag_q = False
        for j in question_words:
            if j in i:
                question.append(i)
                flag_q = True
                break
        if flag_q == False:
            statements.append(i)

    if len(question)==0:
        question.append(microstatements[-1])
        statements = statements[:len(statements)-1]
    # question = microstatements[-1]
    # statements = microstatements[:len(microstatements)-1]
    # for i in microstatements:
    #     if isQuestion_DL(i):
    #         question.append(i)
    #     else:
    #         statements.append(i)  
    return {"question":question,"statements":statements}

In [12]:
def clean_mwp(mwp):
        """
	
		converts to Lowercase
		changes number names to numbers
		"""
        mwp =  mwp.lower()
        #mwp = spelling_correction(mwp)
        mwp = convertNumberNames(mwp)
        mwp = appendQuantities(mwp)
        #print(mwp)
        return mwp
    
def resolve_coref(sentence):
        """ 
		Neural coref resolution function
		@input : sentence without coref resolution
		@output : coref resolved sentence 
		"""
        doc = nlp_pronoun(sentence)
        #self.mwp = doc._.coref_resolved
        return doc._.coref_resolved
def get_microstatements(mwp):
        micros = []
        mwp = clean_mwp(mwp)
        #self.resolve_coref()
        for sent in mwp.split('.'):
            micros.extend(extract_microstatements(sent))
            # print('For sentence:\n', sent, '\nMS:\n', micros)
        final_micros = []
        for i in micros:
            i = re.sub(r'[^\w\s]', '', i)
            final_micros.append(i)
        quesornot = identify_question(final_micros)
        #print(quesornot)
        body_string = ""
        for i in quesornot['statements']:
            body_string = body_string + i + "."
        print('---------before neuralcoref------------------')
        #body_string = resolve_coref(body_string)
        final_micros = body_string[:-1].split(".")
        final_micros.append(quesornot['question'][0])
        #print(final_micros)
        return final_micros

In [13]:
for i in sample_test_data:
  print(get_microstatements(i))

---------before neuralcoref------------------
[' peter ', ' ']
---------before neuralcoref------------------
[' william ', ' ']
---------before neuralcoref------------------
[' william ', ' ']
---------before neuralcoref------------------
[' william ', ' ']
---------before neuralcoref------------------
[' dad ', ' ']
---------before neuralcoref------------------
[' mom ', ' ']
---------before neuralcoref------------------
[' william ', ' ']
---------before neuralcoref------------------
[' william ', ' ']
---------before neuralcoref------------------
[' mom ', ' ']
---------before neuralcoref------------------
[' mary ', ' ']


In [20]:
for i in sample_test_data:
  print(i)
  print(extract_microstatements(i.lower()))

Peter and Mary had 2 sons and 6 daughters.
[' peter ']
William and Kate had 2 sons and 6 daughters.
[' william ']
William and Elizabeth had 2 boys and 6 girls.
[' william ']
William and Mary had 2 daughters and 6 sons.
[' william ']
Dad and Mom had 2 children and 6 grandchildren.
[' dad ']
Mom and Dad had 2 boys and 6 girls.
[' mom ']
William and Mary had 2 sons and 6 children.
[' william ']
William and Elizabeth had 2 sons and 6 grandchildren.
[' william ']
Mom and Dad had 2 brothers and 6 sisters.
[' mom ']
Mary and Joseph had 2 sons and 6 daughters.
[' mary ']
